In [8]:
import requests
import json
from dotenv import load_dotenv
import os


load_dotenv()

def fetch_traffic_reports():
    # URL API
    url = 'https://api.navitia.io/v1/coverage/fr-idf/line_reports'

    # clé api from env
    api_key = os.getenv('token_navitia')
    headers = {
        'Authorization': api_key
    }
    
    # requete
    response = requests.get(url, headers=headers)

  
    if response.status_code == 200:
       
        data = response.json()
        # ecriture requete positif en hson
        with open('line_reports.json', 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print("Data written to line_reports.json")
    else:
        
        print('Failed to retrieve data: ', response.status_code)

# Appel func
fetch_traffic_reports()


Data written to line_reports.json


In [18]:
import pandas as pd
from datetime import datetime

with open('line_reports.json') as f:
    data = json.load(f)

disruptions_data = []

for disruption in data['disruptions']:
    for impacted_object in disruption['impacted_objects']:
        pt_object = impacted_object['pt_object']
        # Check 'stop_point' key existe 
        
        if 'stop_point' in pt_object:  
            stop_point = pt_object['stop_point']
            location_name = stop_point['name']
            location_label = stop_point['label']
            longitude = stop_point['coord']['lon']
            latitude = stop_point['coord']['lat']
            address_label = stop_point['address']['label']
            fare_zone = stop_point['fare_zone']['name']
            transport_type = pt_object['embedded_type']
        else:
            # none si on trouve pas
            location_name = location_label = longitude = latitude = address_label = fare_zone = transport_type = None
        
        # check pour le mode de transport quand il se trouve dedans
        if 'line' in pt_object and 'physical_modes' in pt_object['line']:
            physical_mode = pt_object['line']['physical_modes'][0]['name']
        else:
            physical_mode = None

        disruption_info = {
            "disruption_id": disruption['id'],
            "status": disruption['status'],
            "cause": disruption['cause'],
            "category": disruption['category'],
            "severity_name": disruption['severity']['name'],
            "severity_effect": disruption['severity']['effect'],
            "severity_color": disruption['severity']['color'],
            "severity_priority": disruption['severity']['priority'],
            "message_text": disruption['messages'][0]['text'],
            "application_period_begin": datetime.strptime(disruption['application_periods'][0]['begin'], '%Y%m%dT%H%M%S'),
            "application_period_end": datetime.strptime(disruption['application_periods'][0]['end'], '%Y%m%dT%H%M%S'),
            "location_name": location_name,
            "location_label": location_label,
            "longitude": longitude,
            "latitude": latitude,
            "address_label": address_label,
            "fare_zone": fare_zone,
            "transport_type": transport_type,
            "physical_mode": physical_mode
        }
        
        disruptions_data.append(disruption_info)

# dataframe gang
df_disruptions = pd.DataFrame(disruptions_data)

df_disruptions

,disruption_id,status,cause,category,severity_name,severity_effect,severity_color,severity_priority,message_text,application_period_begin,application_period_end,location_name,location_label,longitude,latitude,address_label,fare_zone,transport_type,physical_mode
0,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,Clément Ader,Clément Ader (Argenteuil),2.24347,48.942583,Rue de la Tour Billy (Argenteuil),4,stop_point,None
1,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,Charles de Gaulle - Henri Barbusse,Charles de Gaulle - Henri Barbusse (Argenteuil),2.245179,48.940498,Avenue du Général de Gaulle (Argenteuil),4,stop_point,None
2,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,Place du 11 Novembre,Place du 11 Novembre (Argenteuil),2.239094,48.936521,Nouvelle Impasse Duguay (Argenteuil),4,stop_point,None
3,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,Avenue du Château,Avenue du Château (Argenteuil),2.233204,48.933282,Allée Simone Veil (Argenteuil),4,stop_point,None
4,e9cbe72a-04c5-11ef-9db7-0a58a9feac02,active,travaux,Incidents,bloquante,NO_SERVICE,#FF0000,0,<p>La ligne 272 est déviée : les arrêts situés...,2024-04-17 06:00:00,2024-06-01 03:45:00,None,None,None,None,None,None,None,Bus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,f7588214-069a-11ef-98d5-0a58a9feac02,active,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,"<p><span style=""background-color:rgb(255,255,2...",2024-04-29 17:15:00,2024-04-30 23:59:00,None,None,None,None,None,None,None,Bus
100,f7588214-069a-11ef-98d5-0a58a9feac02,active,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,"<p><span style=""background-color:rgb(255,255,2...",2024-04-29 17:15:00,2024-04-30 23:59:00,None,None,None,None,None,None,None,Bus
101,f7588214-069a-11ef-98d5-0a58a9feac02,active,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,"<p><span style=""background-color:rgb(255,255,2...",2024-04-29 17:15:00,2024-04-30 23:59:00,None,None,None,None,None,None,None,Bus
102,f7588214-069a-11ef-98d5-0a58a9feac02,active,perturbation,Incidents,perturbée,SIGNIFICANT_DELAYS,#EF662F,30,"<p><span style=""background-color:rgb(255,255,2...",2024-04-29 17:15:00,2024-04-30 23:59:00,None,None,None,None,None,None,None,Bus
